##### my-zero-to-gbm-proj-assign

## Optiver Realized Volatility Prediction

This dataset contains stock market data relevant to the practical execution of trades in the financial markets. In particular, it includes order book snapshots and executed trades. With one second resolution, it provides a uniquely fine grained look at the micro-structure of modern financial markets.

This is a code competition where only the first few rows of the test set are available for download. The rows that are visible are intended to illustrate the hidden test set format and folder structure. The remainder will only be available to your notebook when it is submitted. The hidden test set contains data that can be used to construct features to predict roughly 150,000 target values. Loading the entire dataset will take slightly more than 3 GB of memory, by our estimation.

This is also a forecasting competition, where the final private leaderboard will be determined using data gathered after the training period closes, which means that the public and private leaderboards will have zero overlap. During the active training stage of the competition a large fraction of the test data will be filler, intended only to ensure the hidden dataset has approximately the same size as the actual test data. The filler data will be removed entirely during the forecasting phase of the competition and replaced with real

In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import jovian

<IPython.core.display.Javascript object>

#### Data Description

book_[train/test].parquet A parquet file partitioned by stock_id. Provides order book data on the most competitive buy and sell orders entered into the market. The top two levels of the book are shared. The first level of the book will be more competitive in price terms, it will then receive execution priority over the second level.

stock_id - ID code for the stock. Not all stock IDs exist in every time bucket. Parquet coerces this column to the categorical data type when loaded; you may wish to convert it to int8.
time_id - ID code for the time bucket. Time IDs are not necessarily sequential but are consistent across all stocks.
seconds_in_bucket - Number of seconds from the start of the bucket, always starting from 0.
bid_price[1/2] - Normalized prices of the most/second most competitive buy level.
ask_price[1/2] - Normalized prices of the most/second most competitive sell level.
bid_size[1/2] - The number of shares on the most/second most competitive buy level.
ask_size[1/2] - The number of shares on the most/second most competitive sell level.
trade_[train/test].parquet A parquet file partitioned by stock_id. Contains data on trades that actually executed. Usually, in the market, there are more passive buy/sell intention updates (book updates) than actual trades, therefore one may expect this file to be more sparse than the order book.

stock_id - Same as above.
time_id - Same as above.
seconds_in_bucket - Same as above. Note that since trade and book data are taken from the same time window and trade data is more sparse in general, this field is not necessarily starting from 0.
price - The average price of executed transactions happening in one second. Prices have been normalized and the average has been weighted by the number of shares traded in each transaction.
size - The sum number of shares traded.
order_count - The number of unique trade orders taking place.
train.csv The ground truth values for the training set.

stock_id - Same as above, but since this is a csv the column will load as an integer instead of categorical.
time_id - Same as above.
target - The realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. There is no overlap between feature and target data. You can find more info in our tutorial notebook.
test.csv Provides the mapping between the other data files and the submission file. As with other test files, most of the data is only available to your notebook upon submission with just the first few rows available for download.

stock_id - Same as above.
time_id - Same as above.
row_id - Unique identifier for the submission row. There is one row for each existing time ID/stock ID pair. Each time window is not necessarily containing every individual stock.
sample_submission.csv - A sample submission file in the correct format.

row_id - Same as in test.csv.
target - Same definition as in train.csv. The benchmark is using the median target value from train.csv.

In [ ]:
# Execute this to save new versions of the notebook
#jovian.commit(project="my-zero-to-gbm-proj-assign")
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "arun-gansi/my-zero-to-gbm-proj-assign-da8f6" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/arun-gansi/my-zero-to-gbm-proj-assign-da8f6


'https://jovian.ai/arun-gansi/my-zero-to-gbm-proj-assign-da8f6'

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('../../../data/optiver-realized-volatility-prediction/train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


## Points to be addressed before ML modeling

1. stock_id - ID code for the stock. <font color='red'> Not all stock IDs exist in every time bucket </font>. Parquet coerces this column to the categorical data type when loaded;  <font color='red'> you may wish to convert it to int8 </font>
2. We have missing “seconds_in_bucket” field?.A: That means there is no related market activities during the last second. For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, <font color='red'> you can forward fill the missing data point for all field in book data.</font>
3. I'm trying to make trade data fixed sized. Since missing seconds_in_bucket implies no trade happening within that one-second window, is it technically correct to resample trade data to 600 seconds and fill it with zeros?. Hi, it is correct to assume 0 for order count and size. Some assumptions are required for the price, though. A price of 0 might cause issues.
4. the trade data at T seconds contains a 1-second aggregation of executed orders between [T, T+1 second]
5. One time_id represents a unique 20-minutes trading window which is consistent across all stocks As an example, let’s say time_id = 1 is representing a window between 1900-01-01 12:00:00 and 1900-01-01 12:20:00, then the book data of all stocks for that time_id are is taken from the same window. The data in the first 10 minutes window is shared with all of you, while the order book data of the second 10-minutes is used to build the target for you to predict. The dataset is rolling in such a way that feature and target data will always have zero overlap. Note that time_id is randomly shuffled, so it will not contain any information other than serving as a bridge between different dataset.
6. We can demonstrate the data structure in below way:
<img src = https://www.optiver.com/wp-content/uploads/2021/05/DataBucketing.png>
7. In our competition, we shared the last snapshot of order book for each second. Imagine you have a time_id starting from 1900-01-01 12:00:00, the book update data on seconds_in_bucket = 1 represents the last snapshot of order book update between 12:00:00 and 12:00:01. Similarly to order book data in terms of granularity, but the trade data represents the aggregation of all individual orders happened within one second.
8. So per stock, under the same time_id, the trade data on seconds_in_bucket = 1 represents the aggregation of all individual executed orders between 12:00:00 and 12:00:01. The size is the sum of the size in each individual order, while the price is aggregated as a volume weighted average price of all trades. A straightforward WAP formula can be found on Investopedia.
9. Q: Why we have missing “seconds_in_bucket” field?
A: That means there is no related market activities during the last second.

For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, you can forward fill the missing data point for all field in book data. For trade data, it implies no trade happening within that one-second window. One thing to note that trade data tends to be more sparse than book data in many cases.

10. 


Taking the first row of data, it implies that the realized vol of the target bucket for time_id 5, stock_id 0 is 0.004136. How does the book and trade data in feature bucket look like for us to build signals?

In [ ]:
book_example = pd.read_parquet('../../../data/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('../../../data/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

##### Book data snapshot

In [ ]:
book_example['time_id'].unique()
book_example.head(3)
# book_example.shape

##### Trade date snapshot

In [ ]:
trade_example.head()

##### Realized volatility calculation

our target is to predict short-term realized volatility. Although the order book and trade data for the target cannot be shared, we can still present the realized volatility calculation using the feature data we provided.

As realized volatility is a statistical measure of price changes on a given stock, to calculate the price change we first need to have a stock valuation at the fixed interval (1 second). We will use weighted averaged price, or WAP, of the order book data we provided.

In [ ]:
book_example['bid_price'] = (book_example['bid_price1'] * book_example['bid_size1'] + book_example['bid_price2'] * book_example['bid_size2'])/(book_example['bid_size1'] + book_example['bid_size2'])
book_example['bid_size'] = (book_example['bid_size1'] + book_example['bid_size2'])

book_example['ask_price'] = (book_example['ask_price1'] * book_example['ask_size1'] + book_example['ask_price2'] * book_example['ask_size2'])/(book_example['ask_size1'] + book_example['ask_size2'])
book_example['ask_size'] = (book_example['ask_size1'] + book_example['ask_size2'])

book_example['wap'] = (book_example['bid_price'] * book_example['ask_size'] + book_example['ask_price'] * book_example['bid_size']) / (book_example['bid_size'] +  book_example['ask_size'])

book_example['wap1'] = (book_example['bid_price1'] * book_example['ask_size1'] + book_example['ask_price1'] * book_example['bid_size1']) / (book_example['bid_size1'] +  book_example['ask_size1'])

book_example['wap2'] = (book_example['bid_price2'] * book_example['ask_size2'] + book_example['ask_price2'] * book_example['bid_size2']) / (book_example['bid_size2'] +  book_example['ask_size2'])

The WAP of the stock is plotted below

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

### Log returns

How can we compare the price of a stock between yesterday and today?

The easiest method would be to just take the difference. This is definitely the most intuitive way, however price differences are not always comparable across stocks. For example, let's assume that we have invested \$ 1000 in both stock A and stock B and that stock A moves from  \$ 100 to  \$ 102 and stock B moves from  \$ 10 to  \$ 11. We had a total of 10 shares of A ( \$1000 / \$100=10 ) which led to a profit of  10⋅(\$102−\$100)=\$20  and a total of 100 shares of B that yielded \$100. So the price increase was larger for stock A, although the move was proportionally much larger for stock B.

We can solve the above problem by dividing the move by the starting price of the stock, effectively computing the percentage change in price, also known as the stock return. In our example, the return for stock A was  \$102−\$100/\$100=2% , while for stock B it was  \$11−\$10/\$10=10% . The stock return coincides with the percentage change in our invested capital.

Returns are widely used in finance, however log returns are preferred whenever some mathematical modelling is required. Calling  St  the price of the stock  S  at time  t , we can define the log return between  t1  and  t2  as:

 - r(t1,t2)=log(St2/St1)
 
Usually, we look at log returns over fixed time intervals, so with 10-minute log return we mean  rt=rt−10min,t .

Log returns present several advantages, for example:

- they are additive across time  r(t1,t2) + r(t2,t3) = (rt1,t3) 
- regular returns cannot go below -100%, while log returns are not bounded

Next we will compute the log return

To compute the Log Return, we will simply take the logarithm of the ratio between two consecutive WAP. The first row will have an empty return as the previous book update is unknown, therefore the empty return data point will be dropped.

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])

#~ means NOT 
# also the null condition is applied because when we .diff() method the first row will have empty value as we cannot have diff of return with respect to the previous time id as the first row is the very first time id
book_example = book_example[~book_example['log_return'].isnull()]

In [ ]:
book_example['log_return'].isnull().sum()

Let's plot the tick-to-tick return of this instrument over this time bucket

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

###Realized volatility

When we trade options, a valuable input to our models is the standard deviation of the stock log returns. The standard deviation will be different for log returns computed over longer or shorter intervals, for this reason it is usually normalized to a 1-year period and the annualized standard deviation is called volatility.

In this competition, you will be given 10 minutes of book data and we ask you to predict what the volatility will be in the following 10 minutes. Volatility will be measured as follows:

We will compute the log returns over all consecutive book updates and we define the realized volatility,  σ , as the squared root of the sum of squared log returns.

\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}
 
Where we use WAP as price of the stock to compute log returns.

We want to keep definitions as simple and clear as possible, so that Kagglers without financial knowledge will not be penalized. So we are not annualizing the volatility and we are assuming that log returns have 0 mean.


The realized vol of stock 0 in this feature bucket, will be:

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

### Naive prediction: using past realized volatility as target

A commonly known fact about volatility is that it tends to be autocorrelated. We can use this property to implement a naive model that just "predicts" realized volatility by using whatever the realized volatility was in the initial 10 minutes.

Let's calculate the past realized volatility across the training set to see how predictive a single naive signal can be.

In [4]:
import glob
list_order_book_file_train = glob.glob('../../../data/optiver-realized-volatility-prediction/book_train.parquet/*')
#list_order_book_file_train

As the data is partitioned by stock_id (each stock id is one folder), we try to calculcate realized volatility stock by stock and combine them into one target file. Note that the stock id as the partition column is not present if we load the single file so we will remedy that manually. We will reuse the log return and realized volatility functions defined in the previous session.

In [ ]:
def compute_wap(df_stock_book):
       df_stock_book['bid_price'] = (df_stock_book['bid_price1'] * df_stock_book['bid_size1'] + df_stock_book['bid_price2'] * df_stock_book['bid_size2'])/(df_stock_book['bid_size1'] + df_stock_book['bid_size2'])
       df_stock_book['bid_size'] = (df_stock_book['bid_size1'] + df_stock_book['bid_size2'])

       df_stock_book['ask_price'] = (df_stock_book['ask_price1'] * df_stock_book['ask_size1'] + df_stock_book['ask_price2'] * df_stock_book['ask_size2'])/(df_stock_book['ask_size1'] + df_stock_book['ask_size2'])
       df_stock_book['ask_size'] = (df_stock_book['ask_size1'] + df_stock_book['ask_size2'])

       df_stock_book['wap'] = (df_stock_book['bid_price'] * df_stock_book['ask_size'] + df_stock_book['ask_price'] * df_stock_book['bid_size']) / (df_stock_book['bid_size'] +  df_stock_book['ask_size'])
       return df_stock_book['wap']

In [ ]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
#    df_book_data['wap'] = (df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
#                                     df_book_data['bid_size1']+ df_book_data[
#                                  'ask_size1'])
    df_book_data['wap'] = compute_wap(df_book_data)
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
df_order_book = pd.DataFrame()
for file in list_order_book_file_train:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book = pd.concat([df_order_book,df_stock_book])


In [5]:
import random
# # Get current state and store
# state = random.getstate()
# # set current state
# random.setstate(state)
random.seed(42)
list_order_book_file_train_50 = random.sample(list_order_book_file_train, 50)
list_order_book_file_train_50_old = list_order_book_file_train_50
random.seed(42)
list_order_book_file_train_50 = random.sample(list_order_book_file_train, 50)
assert(list_order_book_file_train_50 == list_order_book_file_train_50_old)
#list(zip(list_order_book_file_train_50,list_order_book_file_train_50_old))

In [6]:
df_order_book_50 = pd.DataFrame()

for file in list_order_book_file_train_50:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book_50 = pd.concat([df_order_book_50,df_stock_book])


In [9]:
len(list_order_book_file_train_50)

df_order_book_50.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.000354,1.000472,1.000236,1.000591,101,40,201,321,68
1,5,1,1.000354,1.000591,1.000236,1.000827,301,100,201,100,68
2,5,2,1.000354,1.000591,1.000236,1.000827,201,100,301,100,68
3,5,3,1.000236,1.000591,1.000118,1.000827,301,200,201,100,68
4,5,4,1.000354,1.000591,1.000236,1.000827,1,300,483,100,68


In [10]:
#There are 112 unique stcoks are there in th order book
len(df_order_book_50.stock_id.unique())

50

In [11]:
grouped = df_order_book_50.groupby('time_id')

In [12]:
##The no of stocks being traded at a given time id is not the same across the time ids. This is because not all stocks are actively traded in all time ids
# There are 28 TimeIds where some stocks are not traded at all. This is out of total 3830 TimeIds
a_lt_50 = grouped.stock_id.nunique() < 50

stock_short_timeid = grouped.stock_id.unique()[list(a_lt_50.values)]

In [13]:
stock_short_timeid.shape

(27,)

In [14]:
list(df_order_book_50.index)[-1]

1834155

In [15]:
#grouped.filter(a_lt_112)
#len(df_order_book_50.loc[df_order_book_50.seconds_in_bucket == 0])
print(df_order_book_50.time_id.nunique(),df_order_book_50.stock_id.nunique(),df_order_book_50.seconds_in_bucket.nunique())
#There are a total of 3830 time series, 112 stocks and 600 seconds bucket 
# In this 3830 Times Ids there are a maximum possible order booking of 3830*600 = 2,298,000. But the actual is  

3830 50 600


In [7]:
# with open('book_index.csv','w+') as f:
#     for items in list(df_order_book_50.index):
#         f.write('%s\n' %items)
#new_index
df_order_book_50.reset_index(inplace=True)
#df_order_book_50.head(-1)
#df_order_book_50[df_order_book_50.index != df_order_book_50['index']]
#(72311913, 11)

In [8]:
df_order_book_50_idxed = df_order_book_50.copy()

In [9]:
new_index = pd.Index(np.arange(0,600), name="seconds_in_bucket")

In [13]:
df_order_book_50_idxed.astype({'stock_id':np.int8})

,index,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,0,5,0,1.000354,1.000472,1.000236,1.000591,101,40,201,321,68
1,1,5,1,1.000354,1.000591,1.000236,1.000827,301,100,201,100,68
2,2,5,2,1.000354,1.000591,1.000236,1.000827,201,100,301,100,68
3,3,5,3,1.000236,1.000591,1.000118,1.000827,301,200,201,100,68
4,4,5,4,1.000354,1.000591,1.000236,1.000827,1,300,483,100,68
...,...,...,...,...,...,...,...,...,...,...,...,...
72311908,1834151,32767,593,1.002166,1.003610,1.000722,1.005054,52200,66235,82550,64431,31
72311909,1834152,32767,594,1.002166,1.003610,1.000722,1.005054,53700,66235,81050,64431,31
72311910,1834153,32767,595,1.002166,1.003610,1.000722,1.005054,52200,66235,82550,64431,31
72311911,1834154,32767,596,1.002166,1.003610,1.000722,1.005054,55700,66235,81050,64431,31


In [14]:
df_order_book_50_idxed.dtypes

index                  int64
time_id                int16
seconds_in_bucket      int16
bid_price1           float32
ask_price1           float32
bid_price2           float32
ask_price2           float32
bid_size1              int32
ask_size1              int32
bid_size2              int32
ask_size2              int32
stock_id              object
dtype: object

In [15]:
df_order_book_50_idxed.reindex( pd.MultiIndex.from_product([df_order_book_50_idxed['time_id'],df_order_book_50_idxed['stock_id'],new_index], names=['time_id', 'stock_id','seconds_in_bucket']))

MemoryError: Unable to allocate 5.44 EiB for an array with shape (3137407657031741400,) and data type int16

In [ ]:
#for a given time_id and stiock_id itself, the number of "seconds_in_bucket" is not the same. THis is the same across
df_order_book_50.groupby(['time_id','stock_id']).seconds_in_bucket.count()

Looping through each individual stocks, we can get the past realized volatility as prediction for each individual stocks.

In [ ]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [ ]:
df_past_realized_train.head()

Let's join the output dataframe with train.csv to see the performance of the naive prediction on training set.

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

We will evaluate the naive prediction result by two metrics: RMSPE and R squared.

In [ ]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')